# Лекция 3.4. Matplotlib, Seaborn и Plotly. Визуальный анализ данных на реальном примере

Произведем импорт библиотек и настройку параметров графиков

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly
import plotly.graph_objs as go

#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 

#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 8, 5

Считаем в DataFrame знакомые нам данные по оттоку клиентов телеком-оператора.


In [0]:
df = pd.read_csv('https://github.com/Eductorium/DataScience/raw/master/Module2/data/telecom_churn.csv')

Проверим, все ли нормально считалось – посмотрим на первые 5 строк (метод head).

In [0]:
df.head()

Число строк (клиентов) и столбцов (признаков):

In [0]:
df.shape

Посмотрим на признаки и убедимся, что пропусков ни в одном из них нет – везде по 3333 записи. В случае пропусков, мы можем оставить только записи без пропуском с помощью функции df.dropna()

In [0]:
df.info()

**Описание признаков:**

![](https://github.com/Eductorium/DataScience/raw/master/Module2/img/params.png)

Целевая переменная: **Churn** – Признак оттока, бинарный признак (1 – потеря клиента, то есть отток). Потом мы будем строить модели, прогнозирующие этот признак по остальным, поэтому мы и назвали его целевым.

Посмотрим на распределение целевого класса – оттока клиентов.

In [0]:
df['Churn'].value_counts()

In [0]:
df['Churn'].value_counts().plot(kind='bar', label='Churn')
plt.legend()
plt.title('Распределение оттока клиентов');

Выделим следующие группы признаков (среди всех кроме Churn ):
* бинарные: International plan, Voice mail plan
* категориальные: State, Area code
* порядковые: Customer service calls
* количественные: все остальные

## Анализ количественных и порядковых признаков


Посмотрим на корреляции количественных признаков.

In [0]:
corr_matrix = df.drop(['International plan', 'Voice mail plan', 'State', 
                      'Area code', 'Customer service calls', 'Churn'], axis=1).corr()

In [0]:
sns.heatmap(corr_matrix);

По раскрашенной матрице корреляций видно, что такие признаки как Total day charge считаются по проговоренным минутам (Total day minutes). То есть 4 признака (Total day charge, Total eve charge, Total night charge, Total intl charge) можно выкинуть, они не несут полезной информации, поскольку зависят от других признаков - Total day minutes, Total eve minutes, Total night minutes и Total intl minutes.

Теперь посмотрим на распределения всех интересующих нас количественных признаков, а также категориальных (признак - *Customer service calls*). На бинарные/категориальные признаки будем смотреть отдельно.

In [0]:
features = list(set(df.columns) - set(['International plan', 'Voice mail plan', 'State', 'Area code',
                                      'Total day charge',   'Total eve charge', 'Total night charge',
                                        'Total intl charge', 'Customer service calls', 'Churn']))

df[features].hist(figsize=(14,12));

Видим, что большинство признаков распределены нормально. Исключения – число звонков в сервисный центр (*Customer service calls*) (тут больше подходит пуассоновское распределение) и число голосовых сообщений (*Number vmail messages*, пик в нуле, т.е. это те, у кого голосовая почта не подключена). Также смещено распределение числа международных звонков (*Total intl calls*).

Еще полезно строить вот такие картинки, где на главной диагонали рисуются распределения признаков, а вне главной диагонали – диаграммы рассеяния для пар признаков. Бывает, что это приводит к каким-то выводам, но в данном случае все примерно понятно, без сюрпризов.

In [0]:
# c svg pairplot браузер начинает тормозить
%config InlineBackend.figure_format = 'png'
sns.pairplot(df[features + ['Churn']], hue='Churn');

**Дальше посмотрим, как признаки связаны с целевым – с оттоком.**

Построим boxplot-ы, описывающее статистики распределения количественных признаков в двух группах: среди лояльных и ушедших клиентов.

In [0]:
%config InlineBackend.figure_format = 'svg' 

fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(16, 10))

for idx, feat in  enumerate(features):
    sns.boxplot(x='Churn', y=feat, data=df, ax=axes[int(idx / 4), idx % 4])
    axes[int(idx / 4), idx % 4].legend()
    axes[int(idx / 4), idx % 4].set_xlabel('Churn')
    axes[int(idx / 4), idx % 4].set_ylabel(feat);

На глаз наибольшее отличие мы видим для признаков *Total day minutes*, *Customer service calls* и *Number vmail messages*. Впоследствии мы научимся определять важность признаков в задаче классификации с помощью случайного леса (или градиентного бустинга), и окажется, что первые два – действительно очень важные признаки для прогнозирования оттока.

Посмотрим отдельно на картинки с распределением кол-ва проговоренных днем минут среди лояльных/ушедших. Слева - знакомые нам боксплоты, справа – сглаженные гистограммы распределения числового признака в двух группах (скорее просто красивая картинка, все и так понятно по боксплоту).

In [0]:
_, axes = plt.subplots(1, 2, sharey=True, figsize=(16, 6))

sns.boxplot(x='Churn', y='Total day minutes', data=df, ax=axes[0]);
sns.violinplot(x='Churn', y='Total day minutes', data=df, ax=axes[1]);

Интересное **наблюдение**: в среднем ушедшие клиенты больше пользуются связью. Возможно, они недовольны тарифами, и одной из мер борьбы с оттоком будет понижение тарифных ставок (стоимости мобильной связи). Но это уже компании надо будет проводить дополнительный экономический анализ, действительно ли такие меры будут оправданы.

Теперь изобразим распределение числа обращений в сервисный центр (такую картинку мы строили в первой статье). Тут уникальных значений признака не много (признак можно считать как количественным целочисленным, так и порядковым), и наглядней изобразить распределение с помощью countplot.

In [0]:
sns.countplot(x='Customer service calls', hue='Churn', data=df);


**Наблюдение:** доля оттока сильно возрастает начиная с 4 звонков в сервисный центр.

Если же построим boxplot, то также увидим, что с ростом числа обращений в сервисныйцентр, отток увеличивается. Но на предыдущей диаграмме было конкретно видно, что это происходит, начиная с 4 звонка.

In [0]:
sns.boxplot(x='Churn', y='Customer service calls', data=df);

Построим график для последнего из выделенных количественных признаков - *Number vmail messages*.

In [0]:
sns.boxplot(x='Churn', y='Number vmail messages', data=df);

По графику можно сделать вывод о том, что большинство ушедших клиентов не пользовались голосовой почтой - не отправляли сообщения, хотя услуга могла быть и подключена.

## Анализ бинарных признаков

Теперь посмотрим на связь бинарных признаков International plan и Voice mail plan с оттоком.

In [0]:
_, axes = plt.subplots(1, 2, sharey=True, figsize=(16,6))

sns.countplot(x='International plan', hue='Churn', data=df, ax=axes[0]);
sns.countplot(x='Voice mail plan', hue='Churn', data=df, ax=axes[1]);

**Наблюдение:** когда роуминг подключен, доля оттока намного выше, т.е. наличие международного роуминга – сильный признак. Про голосовую почту такого нельзя сказать.


## Анализ категориальных признаков

Наконец, посмотрим, как с оттоком связан категориальный признак *State*. С ним уже не так приятно работать, поскольку число уникальных штатов довольно велико – 51. Можно в начале построить сводную табличку или посчитать процент оттока для каждого штата. Но данных по каждом штату по отдельности маловато (ушедших клиентов всего от 3 до 17 в каждом штате), поэтому, возможно, признак *State* впоследствии не стоит добавлять в модели классификации из-за риска переобучения (но мы это будем проверять на кросс-валидации, stay tuned!).

Доли оттока для каждого штата:

In [0]:
df.groupby(['State'])['Churn'].agg([np.mean]).sort_values(by='mean', ascending=False).T

Видно, что в Нью-Джерси и Калифорнии доля оттока выше 25%, а на Гавайях и в Аляске меньше 5%. Но эти выводы построены на слишком скромной статистике и возможно, это просто особенности имеющихся данных.

На этом все, следующим этапом будет выполнение домашнего задания.